# End-to-End TensorFlow Pipeline Notebook

Dieses Notebook führt dich Schritt für Schritt durch einen kompletten Machine-Learning-Workflow mit TensorFlow:

1. **Daten-Pipeline**: Laden, Vorverarbeiten und Batchen von Bildern mit `tf.data`.
2. **TFRecord-Erstellung**: Konvertiere deine Bilder und Labels in das effiziente TFRecord-Format.
3. **Basic CNN**: Implementiere ein einfaches Convolutional Neural Network (CNN) von Grund auf.
4. **Pretrained ResNet**: Nutze ein ResNet-Modell mit vortrainierten Gewichten.
5. **Modellspeicherung**: Speichere und lade dein trainiertes Modell.
6. **Evaluation**: Beurteile die Modellleistung auf einem Evaluierungsdatensatz.

Jede Zelle enthält ausführliche Erklärungen und Verweise auf die TensorFlow-Dokumentation.

## 1. Setup und Imports

Wir starten mit den benötigten Bibliotheken. Stelle sicher, dass TensorFlow 2.x installiert ist.

Referenzen:
- TensorFlow Installationsanleitung: https://www.tensorflow.org/install
- TensorFlow 2 Guide: https://www.tensorflow.org/guide

In [ ]:
import tensorflow as tf
import numpy as np
import os
from pathlib import Path
import json

In [1]:
print("Andi ist cool")

Andi ist cool


## 2. Daten-Pipeline erstellen

Wir erstellen zwei separate Datensätze:
- **Trainings-Datensatz**: Zum Trainieren unseres Modells
- **Evaluierungs-Datensatz**: Zum Testen unseres Modells

Die Pipeline lädt Bilder aus Ordnern, verkleinert sie auf eine einheitliche Größe und normalisiert die Pixelwerte.

**Was passiert hier:**
- Bilder werden geladen und in das richtige Format gebracht
- Pixelwerte werden von 0-255 auf 0-1 normalisiert (das hilft beim Training)
- Bilder werden in kleinere Gruppen (Batches) aufgeteilt

**Wichtig:** Wir verwenden hier bewusst einfache Schleifen ohne parallele Verarbeitung, damit der Code leichter zu verstehen ist.

In [ ]:
# =============================================================================
# DATEN-PIPELINE: TRAININGS- UND EVALUIERUNGS-DATENSÄTZE ERSTELLEN
# =============================================================================

# Pfade zu den beiden Datensatz-Ordnern
train_data_dir = Path("./data/uni_test_train_ds")
eval_data_dir = Path("./data/uni_test_eval_ds")

# Parameter für die Bildverarbeitung
batch_size = 4           # Anzahl Bilder pro Batch
img_size = (256, 256)    # Alle Bilder werden auf diese Größe verkleinert

# =============================================================================
# HILFSFUNKTIONEN
# =============================================================================

def load_and_process_image(image_path):
    """
    Lädt ein PNG-Bild und bereitet es für das Training vor.
    """
    # Bild von Festplatte lesen und dekodieren
    image_raw = tf.io.read_file(image_path)
    image = tf.image.decode_png(image_raw, channels=3)
    image = tf.cast(image, tf.float32)
    
    # Auf einheitliche Größe verkleinern und normalisieren
    image = tf.image.resize(image, img_size)
    image = image / 255.0  # Werte von 0-255 auf 0-1 bringen
    
    return image

def load_labels_from_json(labels_file):
    """
    Lädt die Labels aus einer Label Studio JSON-Datei.
    """
    with open(labels_file, 'r') as f:
        label_data = json.load(f)
    
    # Dictionary zum Zuordnen von Bildnamen zu Labels
    image_to_label = {}
    
    for item in label_data:
        # Bildpfad extrahieren
        image_path = item['data']['image']
        filename = image_path.split('/')[-1]  # Nur Dateiname
        
        # Label extrahieren
        if item['annotations'] and len(item['annotations']) > 0:
            annotation = item['annotations'][0]
            if annotation['result'] and len(annotation['result']) > 0:
                label = annotation['result'][0]['value']['choices'][0]
                image_to_label[filename] = label
    
    return image_to_label

def create_dataset_from_directory(data_dir):
    """
    Erstellt einen TensorFlow-Datensatz aus PNG-Bildern und Label Studio Labels.
    """
    print(f"Erstelle Datensatz aus: {data_dir}")
    
    # Labels aus JSON-Datei laden
    labels_file = data_dir / "labels" / f"labels_{data_dir.name}.json"
    image_to_label = load_labels_from_json(labels_file)
    print(f"Labels geladen: {len(image_to_label)} Einträge")
    
    # Alle PNG-Bilder finden (der Ordner heißt 'images', nicht 'imgs')
    image_dir = data_dir / "images"
    image_paths = list(image_dir.glob("*.png"))
    print(f"Gefundene PNG-Bilder: {len(image_paths)}")
    
    if len(image_paths) == 0:
        print(f"WARNUNG: Keine PNG-Bilder in {image_dir} gefunden!")
        return None, None
    
    # Listen für Bilder und Labels erstellen
    images = []
    labels = []
    
    # Jedes Bild verarbeiten
    for image_path in image_paths:
        filename = image_path.name
        
        # Prüfen ob Label vorhanden ist
        if filename in image_to_label:
            # Bild laden
            image = load_and_process_image(str(image_path))
            images.append(image)
            
            # Label hinzufügen
            label = image_to_label[filename]
            labels.append(label)
    
    print(f"Bilder mit Labels verarbeitet: {len(images)}")
    
    if len(images) == 0:
        print("FEHLER: Keine Bilder mit passenden Labels gefunden!")
        return None, None
    
    # In TensorFlow-Tensoren umwandeln
    images_tensor = tf.stack(images)
    
    # Klassen sortieren und in Zahlen umwandeln
    unique_classes = sorted(list(set(labels)))
    print(f"Klassen: {unique_classes}")
    
    label_to_index = {name: i for i, name in enumerate(unique_classes)}
    numeric_labels = [label_to_index[label] for label in labels]
    labels_tensor = tf.constant(numeric_labels)
    
    # TensorFlow-Dataset erstellen und batchen
    dataset = tf.data.Dataset.from_tensor_slices((images_tensor, labels_tensor))
    dataset = dataset.batch(batch_size)
    
    return dataset, unique_classes

# =============================================================================
# DATENSÄTZE ERSTELLEN
# =============================================================================

# Trainings-Datensatz
print("🔄 Erstelle Trainings-Datensatz...")
train_dataset, train_classes = create_dataset_from_directory(train_data_dir)

# Evaluierungs-Datensatz
print("\n🔄 Erstelle Evaluierungs-Dataset...")
eval_dataset, eval_classes = create_dataset_from_directory(eval_data_dir)

# Klassennamen für später speichern und einmal zusammenfassen
class_names = train_classes if train_classes else []
num_classes = len(class_names)

# Zusammenfassung der erstellten Datasets
print(f"\n{'='*60}")
print("📊 DATASET-ÜBERSICHT")
print(f"{'='*60}")
print(f"✅ Trainings-Dataset: {len(train_classes) if train_classes else 0} Klassen")
print(f"✅ Evaluierungs-Dataset: {len(eval_classes) if eval_classes else 0} Klassen")
print(f"🏷️  Erkannte Klassen: {class_names}")
print(f"🔢 Anzahl Klassen: {num_classes}")

# Prüfen ob beide Datasets die gleichen Klassen haben
if train_classes and eval_classes:
    if set(train_classes) == set(eval_classes):
        print("✅ Beide Datasets haben identische Klassen - perfekt!")
    else:
        print("⚠️  WARNUNG: Verschiedene Klassen in Train/Eval Datasets!")
        print(f"   Train: {train_classes}")
        print(f"   Eval:  {eval_classes}")
print(f"{'='*60}")

## 3. TFRecord-Datensatz erstellen

Wir speichern die vorverarbeiteten Bilder und Labels im TFRecord-Format.

- **`tf.train.Example`**: Struktur für einzelne Instanzen
- **`tf.io.TFRecordWriter`**: Schreibt binäre Dateien

Referenz:
- TFRecord Guide: https://www.tensorflow.org/tutorials/load_data/tfrecord

In [ ]:
# =============================================================================
# TFRECORD-DATENSÄTZE ERSTELLEN
# =============================================================================

def _bytes_feature(value):
    """Hilfsfunktion für Bytes-Features."""
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
    """Hilfsfunktion für Integer-Features."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def create_tfrecord(dataset, output_file, dataset_name):
    """
    Erstellt eine TFRecord-Datei aus einem Dataset.
    """
    print(f"Erstelle {dataset_name} TFRecord: {output_file}")
    
    with tf.io.TFRecordWriter(output_file) as writer:
        for images, labels in dataset:
            for img, lbl in zip(images, labels):
                # Bild als PNG kodieren (besser für PNGs als JPEG)
                img_raw = tf.io.encode_png(tf.cast(img * 255, tf.uint8)).numpy()
                
                # Feature erstellen
                feature = {
                    'image_raw': _bytes_feature(img_raw),
                    'label': _int64_feature(int(lbl.numpy()))
                }
                
                # Example erstellen und schreiben
                example = tf.train.Example(features=tf.train.Features(feature=feature))
                writer.write(example.SerializeToString())
    
    print(f"{dataset_name} TFRecord erstellt!")

# TFRecord-Dateien erstellen
train_record_file = './data/train_dataset.tfrecord'
eval_record_file = './data/eval_dataset.tfrecord'

if train_dataset is not None:
    create_tfrecord(train_dataset, train_record_file, "Trainings")

if eval_dataset is not None:
    create_tfrecord(eval_dataset, eval_record_file, "Evaluierungs")

## 4. TFRecord-Dataset laden

Wir laden den TFRecord-Datensatz und wandeln die `Example`-Protobufs zurück in Tensors um.

In [ ]:
# =============================================================================
# TFRECORD-DATASETS LADEN UND FÜR TRAINING VORBEREITEN
# =============================================================================

def parse_tfrecord_example(example_proto):
    """
    Wandelt ein TFRecord-Example zurück in Bild und Label um.
    """
    # Schema definieren
    feature_description = {
        'image_raw': tf.io.FixedLenFeature([], tf.string),
        'label': tf.io.FixedLenFeature([], tf.int64)
    }
    
    # Example parsen
    parsed = tf.io.parse_single_example(example_proto, feature_description)
    
    # Bild dekodieren (PNG, da wir PNG kodiert haben)
    image = tf.io.decode_png(parsed['image_raw'], channels=3)
    image = tf.cast(image, tf.float32)
    image = tf.image.resize(image, img_size) / 255.0
    
    # Label in One-Hot-Encoding umwandeln
    label = tf.one_hot(parsed['label'], depth=num_classes)
    
    return image, label

def load_and_split_tfrecord_dataset(record_file, dataset_name, train_split=0.8):
    """
    Lädt ein TFRecord-Dataset und teilt es in Training und Validation auf.
    """
    print(f"Lade {dataset_name} TFRecord: {record_file}")
    
    # TFRecord-Dataset laden
    raw_dataset = tf.data.TFRecordDataset(record_file)
    
    # Examples parsen
    parsed_dataset = raw_dataset.map(parse_tfrecord_example)
    
    # Dataset-Größe ermitteln (ungefähr)
    dataset_size = sum(1 for _ in parsed_dataset)
    print(f"Dataset-Größe: {dataset_size} Beispiele")
    
    # Dataset shuffeln für bessere Aufteilung
    shuffled_dataset = parsed_dataset.shuffle(buffer_size=dataset_size, seed=42)
    
    # Train/Validation Split
    train_size = int(dataset_size * train_split)
    val_size = dataset_size - train_size
    
    train_dataset = shuffled_dataset.take(train_size)
    val_dataset = shuffled_dataset.skip(train_size)
    
    # Batchen
    train_dataset = train_dataset.batch(batch_size)
    val_dataset = val_dataset.batch(batch_size)
    
    print(f"Training: {train_size} Beispiele ({train_split*100:.0f}%)")
    print(f"Validation: {val_size} Beispiele ({(1-train_split)*100:.0f}%)")
    
    return train_dataset, val_dataset

def load_tfrecord_dataset(record_file, dataset_name):
    """
    Lädt ein TFRecord-Dataset für Evaluation (ohne Split).
    """
    print(f"Lade {dataset_name} TFRecord: {record_file}")
    
    # TFRecord-Dataset laden
    raw_dataset = tf.data.TFRecordDataset(record_file)
    
    # Examples parsen
    parsed_dataset = raw_dataset.map(parse_tfrecord_example)
    
    # Batchen
    batched_dataset = parsed_dataset.batch(batch_size)
    
    return batched_dataset

# =============================================================================
# DATASETS LADEN UND AUFTEILEN
# =============================================================================

# Trainings-TFRecord laden und in Train/Validation aufteilen
train_tfrecord_dataset = None
val_tfrecord_dataset = None
eval_tfrecord_dataset = None

if os.path.exists(train_record_file):
    print("📊 Lade und teile Trainings-Dataset auf...")
    train_tfrecord_dataset, val_tfrecord_dataset = load_and_split_tfrecord_dataset(
        train_record_file, "Trainings", train_split=0.8
    )

# Evaluierungs-TFRecord separat laden (wird nicht zum Training verwendet)
if os.path.exists(eval_record_file):
    print("\n📊 Lade Evaluierungs-Dataset...")
    eval_tfrecord_dataset = load_tfrecord_dataset(eval_record_file, "Evaluierungs")

# Zusammenfassung der verfügbaren Datasets
print(f"\n{'='*60}")
print("📊 VERFÜGBARE DATASETS FÜR TRAINING")
print(f"{'='*60}")
print(f"✅ Training-Dataset: {train_tfrecord_dataset}")
print(f"✅ Validation-Dataset: {val_tfrecord_dataset}")
print(f"📋 Evaluation-Dataset: {eval_tfrecord_dataset} (nur für finale Evaluation)")
print(f"🔢 Anzahl Klassen: {num_classes}")
print(f"🏷️  Klassennamen: {class_names}")
print(f"{'='*60}")

## 5. Basic CNN erstellen und trainieren

Ein einfaches CNN zum Einstieg:

In [ ]:
# =============================================================================
# BASIC CNN ERSTELLEN UND TRAINIEREN
# =============================================================================

# Einfaches CNN-Modell erstellen
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(*img_size, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

# Modell kompilieren
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("Basic CNN Modell-Architektur:")
model.summary()

# Training mit Train/Validation Split (80/20)
epochs = 5

if train_tfrecord_dataset is not None and val_tfrecord_dataset is not None:
    print(f"\n🚀 Trainiere Basic CNN für {epochs} Epochen...")
    print("📊 Training: 80% der Daten")
    print("📊 Validation: 20% der Daten")
    
    history = model.fit(
        train_tfrecord_dataset,
        epochs=epochs,
        validation_data=val_tfrecord_dataset,
        verbose=1
    )
    
    print("✅ Training abgeschlossen!")
else:
    print("❌ Kein Trainings-Dataset verfügbar. Überspringe Training.")

## 6. Pretrained ResNet nutzen

Wir verwenden `tf.keras.applications.ResNet50` mit vortrainierten ImageNet-Gewichten.

Referenz:
- Keras Applications: https://www.tensorflow.org/api_docs/python/tf/keras/applications

In [ ]:
# =============================================================================
# PRETRAINED RESNET NUTZEN
# =============================================================================

# ResNet50 Basis-Modell laden (ohne Top-Layer)
base_model = tf.keras.applications.ResNet50(
    input_shape=(*img_size, 3),
    include_top=False,
    weights='imagenet'
)

# Basis-Modell einfrieren (Transfer Learning)
base_model.trainable = False

# Eigenes Top-Layer hinzufügen
inputs = tf.keras.Input(shape=(*img_size, 3))
x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

resnet_model = tf.keras.Model(inputs, outputs)

# Modell kompilieren
resnet_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("ResNet50 Transfer Learning Modell-Architektur:")
resnet_model.summary()

# Training mit Train/Validation Split (80/20)
if train_tfrecord_dataset is not None and val_tfrecord_dataset is not None:
    print(f"\n🚀 Trainiere ResNet50 für 3 Epochen...")
    print("📊 Training: 80% der Daten")
    print("📊 Validation: 20% der Daten")
    
    resnet_history = resnet_model.fit(
        train_tfrecord_dataset,
        epochs=3,
        validation_data=val_tfrecord_dataset,
        verbose=1
    )
    
    print("✅ ResNet Training abgeschlossen!")
else:
    print("❌ Kein Trainings-Dataset verfügbar. Überspringe Training.")

## 7. Modell speichern und laden

In [ ]:
# =============================================================================
# MODELL SPEICHERN UND LADEN
# =============================================================================

# Ordner für gespeicherte Modelle erstellen
os.makedirs('./saved_models', exist_ok=True)

# Basic CNN Modell speichern
cnn_model_path = './saved_models/basic_cnn.keras'
model.save(cnn_model_path)
print(f"Basic CNN Modell gespeichert unter: {cnn_model_path}")

# ResNet Modell speichern (falls trainiert)
if 'resnet_model' in locals():
    resnet_model_path = './saved_models/resnet_model.keras'
    resnet_model.save(resnet_model_path)
    print(f"ResNet Modell gespeichert unter: {resnet_model_path}")

# Modell wieder laden (Beispiel)
print("\nLade gespeichertes Modell...")
loaded_model = tf.keras.models.load_model(cnn_model_path)
print("Modell erfolgreich geladen!")

# Kurze Übersicht des geladenen Modells
print("\nÜbersicht des geladenen Modells:")
loaded_model.summary()

## 8. Ausführliche Modell-Evaluation

Wir evaluieren beide Modelle (Basic CNN und ResNet) ausführlich mit verschiedenen TensorFlow-Metriken:

- **Accuracy**: Wie oft liegt das Modell richtig?
- **Loss**: Wie sicher ist sich das Modell bei seinen Vorhersagen?
- **Confusion Matrix**: Welche Klassen werden verwechselt?
- **Classification Report**: Detaillierte Metriken pro Klasse

Referenzen:
- TensorFlow Metrics: https://www.tensorflow.org/api_docs/python/tf/keras/metrics
- Model Evaluation: https://www.tensorflow.org/guide/keras/evaluate_and_predict

In [ ]:
# =============================================================================
# MODELL-EVALUATION
# =============================================================================

def evaluate_model(model, dataset, model_name):
    """Einfache und übersichtliche Modell-Evaluation."""
    if dataset is None:
        print(f"❌ {model_name}: Kein Dataset verfügbar")
        return None
    
    # Standard TensorFlow Evaluation
    loss, accuracy = model.evaluate(dataset, verbose=0)
    
    # Vorhersagen sammeln für detaillierte Analyse
    predictions = model.predict(dataset, verbose=0)
    y_pred = tf.argmax(predictions, axis=1)
    
    # Wahre Labels sammeln
    y_true = []
    for _, labels in dataset:
        y_true.extend(tf.argmax(labels, axis=1).numpy())
    
    return {
        'name': model_name,
        'accuracy': accuracy,
        'loss': loss,
        'predictions': y_pred.numpy(),
        'true_labels': y_true
    }

# =============================================================================
# EVALUATION DURCHFÜHREN
# =============================================================================

print("🔍 Modell-Evaluation\n")

# Beide Modelle evaluieren
cnn_results = evaluate_model(model, eval_tfrecord_dataset, "Basic CNN")
resnet_results = evaluate_model(resnet_model, eval_tfrecord_dataset, "ResNet50")

# Ergebnisse anzeigen
results = [cnn_results, resnet_results]
results = [r for r in results if r is not None]  # Nur verfügbare Ergebnisse

if results:
    print(f"{'Modell':<15} {'Accuracy':<12} {'Loss':<10}")
    print("-" * 55)
    
    for result in results:
        acc = f"{result['accuracy']:.3f}"
        loss = f"{result['loss']:.3f}"
        print(f"{result['name']:<15} {acc:<12} {loss:<10}")
    
    # Bestes Modell ermitteln
    best_model = max(results, key=lambda x: x['accuracy'])
    print(f"\n🏆 Bestes Modell: {best_model['name']} ({best_model['accuracy']:.1%} Accuracy)")
    
    # Klassenweise Accuracy (exemplarisch)
    print(f"\n📊 Klassenverteilung (Evaluations-Dataset):")
    for i, class_name in enumerate(class_names):
        count = sum(1 for label in best_model['true_labels'] if label == i)
        print(f"   {class_name}: {count} Samples")

print("\n✅ Evaluation abgeschlossen!")

## 9. Fazit und Ausblick

### Was du gelernt hast:

1. **Daten-Pipeline**: Effizientes Laden und Vorverarbeiten von Bilddaten mit Label Studio JSON-Labels
2. **TFRecord-Format**: Optimierte Datenspeicherung für große Datasets
3. **CNN von Grund auf**: Aufbau eines einfachen Convolutional Neural Networks
4. **Transfer Learning**: Nutzung vortrainierter ResNet50-Gewichte
5. **Modellspeicherung**: Persistierung trainierter Modelle
6. **Umfassende Evaluation**: Professionelle Bewertung mit TensorFlow-Metriken

### Wichtige Erkenntnisse:

- **TFRecord** ermöglicht effizientes Streaming großer Datasets
- **Transfer Learning** kann deutlich bessere Ergebnisse erzielen als Training von Grund auf
- **Evaluation** sollte immer mehrere Metriken und Visualisierungen umfassen
- **Einfacher Code** ist oft besser als komplizierte Parallelverarbeitung (für Lernzwecke)

### Nächste Schritte:

- **Data Augmentation**: Erweitere den Datensatz künstlich
- **Hyperparameter Tuning**: Optimiere Lernrate, Batch-Größe, etc.
- **Andere Architekturen**: Probiere EfficientNet, Vision Transformer

---

### 📚 Weiterführende Ressourcen:

**TensorFlow Grundlagen:**
- [TensorFlow Guide](https://www.tensorflow.org/guide)
- [Keras API](https://www.tensorflow.org/api_docs/python/tf/keras)

**Daten-Pipeline:**
- [tf.data Guide](https://www.tensorflow.org/guide/data)
- [TFRecord Tutorial](https://www.tensorflow.org/tutorials/load_data/tfrecord)

**Computer Vision:**
- [Image Classification Tutorial](https://www.tensorflow.org/tutorials/images/classification)
- [Transfer Learning Guide](https://www.tensorflow.org/tutorials/images/transfer_learning)

**Evaluation & Metriken:**
- [Model Evaluation](https://www.tensorflow.org/guide/keras/evaluate_and_predict)
- [TensorFlow Metrics](https://www.tensorflow.org/api_docs/python/tf/keras/metrics)

**Production Deployment:**
- [TensorFlow Serving](https://www.tensorflow.org/tfx/guide/serving)
- [TensorFlow Lite](https://www.tensorflow.org/lite)

---

### 💡 Tipps für eigene Projekte:

1. **Starte einfach**: Verwende erst kleine Datasets und einfache Modelle
2. **Visualisiere alles**: Confusion Matrix, Trainingsverläufe, Datenverteilung
3. **Dokumentiere**: Notiere Hyperparameter und Ergebnisse
4. **Validiere richtig**: Trenne Training, Validation und Evaluierung strikt
5. **Iteriere**: Verbessere schrittweise statt alles auf einmal zu ändern